In [1]:
# Importing libraries

import pandas as pd
import numpy as np
import os

In [2]:
# Importing data

path = r'C:\Users\nodnarb\Documents\IC Basket Analysis'
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_2.pkl'))

### 1. Mean of order number group by department id

In [4]:
ords_prods_merge.groupby('department_id').agg({'order_number':['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


##### From a quick glance, the mean of the entire dataset is a bit higher than the subset

### 2. Loyalty flag for customers

In [7]:
# Creating max orders column

ords_prods_merge['max_orders'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

C:\Users\nodnarb\AppData\Local\Temp\ipykernel_13948\1718781078.py:3: FutureWarning: The provided callable <function max at 0x000002A4F38AB920> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_orders'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [8]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_time,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_times,max_orders
0,2539329,1,1,2,8,0.0,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Average orders,10
1,2539329,1,1,2,8,0.0,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,both,Mid-range product,Regularly busy,Average orders,10
2,2539329,1,1,2,8,0.0,12427,3,0,Original Beef Jerky,23,19,4.4,both,Low-range product,Regularly busy,Average orders,10
3,2539329,1,1,2,8,0.0,26088,4,0,Aged White Cheddar Popcorn,23,19,4.7,both,Low-range product,Regularly busy,Average orders,10
4,2539329,1,1,2,8,0.0,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,both,Low-range product,Regularly busy,Average orders,10


In [9]:
# Creating loyalty flag from max orders column

ords_prods_merge.loc[ords_prods_merge['max_orders'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [10]:
ords_prods_merge.loc[(ords_prods_merge['max_orders'] <= 40) & (ords_prods_merge['max_orders'] > 10), 'loyalty_flag'] = 'Regular customer'

In [11]:
ords_prods_merge.loc[(ords_prods_merge['max_orders'] <= 10), 'loyalty_flag'] = 'New customer'

In [12]:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

In [13]:
# Grouping loyal customers and prices of products purchased

ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean']})

,prices
,mean
loyalty_flag,
Loyal customer,10.386336
New customer,13.294670
Regular customer,12.495717


##### New customers tend to spend the most while loyal customers spend the least

### 3. Targeting different spenders

In [16]:
# Group by user id and average spending per user

ords_prods_merge['average_spending'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\nodnarb\AppData\Local\Temp\ipykernel_13948\4240252757.py:3: FutureWarning: The provided callable <function mean at 0x000002A4F38BC360> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['average_spending'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [17]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_time,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_times,max_orders,loyalty_flag,average_spending
0,2539329,1,1,2,8,0.0,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Average orders,10,New customer,6.367797
1,2539329,1,1,2,8,0.0,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,both,Mid-range product,Regularly busy,Average orders,10,New customer,6.367797
2,2539329,1,1,2,8,0.0,12427,3,0,Original Beef Jerky,23,19,4.4,both,Low-range product,Regularly busy,Average orders,10,New customer,6.367797
3,2539329,1,1,2,8,0.0,26088,4,0,Aged White Cheddar Popcorn,23,19,4.7,both,Low-range product,Regularly busy,Average orders,10,New customer,6.367797
4,2539329,1,1,2,8,0.0,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,both,Low-range product,Regularly busy,Average orders,10,New customer,6.367797


In [18]:
# Creating spending flag for users

ords_prods_merge.loc[ords_prods_merge['average_spending'] < 10, 'spending_flag'] = 'Low spender'

In [19]:
ords_prods_merge.loc[ords_prods_merge['average_spending'] >= 10, 'spending_flag'] = 'High spender'

In [20]:
ords_prods_merge['spending_flag'].value_counts(dropna = False)

spending_flag
Low spender     31770614
High spender      634245
Name: count, dtype: int64

In [21]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_time,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,department_id,prices,_merge,price_range_loc,busiest_day,busiest_times,max_orders,loyalty_flag,average_spending,spending_flag
0,2539329,1,1,2,8,0.0,196,1,0,Soda,...,7,9.0,both,Mid-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender
1,2539329,1,1,2,8,0.0,14084,2,0,Organic Unsweetened Vanilla Almond Milk,...,16,12.5,both,Mid-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender
2,2539329,1,1,2,8,0.0,12427,3,0,Original Beef Jerky,...,19,4.4,both,Low-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender
3,2539329,1,1,2,8,0.0,26088,4,0,Aged White Cheddar Popcorn,...,19,4.7,both,Low-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender
4,2539329,1,1,2,8,0.0,26405,5,0,XL Pick-A-Size Paper Towel Rolls,...,17,1.0,both,Low-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender


### 4. Frequent vs non-frequent customers

In [23]:
# Grouping user id with days since prior order

ords_prods_merge['median_days_of_prior_order'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

C:\Users\nodnarb\AppData\Local\Temp\ipykernel_13948\2225519363.py:3: FutureWarning: The provided callable <function median at 0x000002A4F39CEE80> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['median_days_of_prior_order'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [24]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_time,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,prices,_merge,price_range_loc,busiest_day,busiest_times,max_orders,loyalty_flag,average_spending,spending_flag,median_days_of_prior_order
0,2539329,1,1,2,8,0.0,196,1,0,Soda,...,9.0,both,Mid-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.0
1,2539329,1,1,2,8,0.0,14084,2,0,Organic Unsweetened Vanilla Almond Milk,...,12.5,both,Mid-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.0
2,2539329,1,1,2,8,0.0,12427,3,0,Original Beef Jerky,...,4.4,both,Low-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.0
3,2539329,1,1,2,8,0.0,26088,4,0,Aged White Cheddar Popcorn,...,4.7,both,Low-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.0
4,2539329,1,1,2,8,0.0,26405,5,0,XL Pick-A-Size Paper Towel Rolls,...,1.0,both,Low-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.0


In [25]:
# Creating flag for customer frequency

ords_prods_merge.loc[ords_prods_merge['median_days_of_prior_order'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [26]:
ords_prods_merge.loc[(ords_prods_merge['median_days_of_prior_order'] > 10) & (ords_prods_merge['median_days_of_prior_order'] <= 20), 'frequency_flag'] = 'Regular customer'

In [27]:
ords_prods_merge.loc[ords_prods_merge['median_days_of_prior_order'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [28]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_time,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,_merge,price_range_loc,busiest_day,busiest_times,max_orders,loyalty_flag,average_spending,spending_flag,median_days_of_prior_order,frequency_flag
0,2539329,1,1,2,8,0.0,196,1,0,Soda,...,both,Mid-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.0,Regular customer
1,2539329,1,1,2,8,0.0,14084,2,0,Organic Unsweetened Vanilla Almond Milk,...,both,Mid-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.0,Regular customer
2,2539329,1,1,2,8,0.0,12427,3,0,Original Beef Jerky,...,both,Low-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.0,Regular customer
3,2539329,1,1,2,8,0.0,26088,4,0,Aged White Cheddar Popcorn,...,both,Low-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.0,Regular customer
4,2539329,1,1,2,8,0.0,26405,5,0,XL Pick-A-Size Paper Towel Rolls,...,both,Low-range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.0,Regular customer


In [29]:
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

frequency_flag
Frequent customer        22796659
Regular customer          6921472
Non-frequent customer     2686728
Name: count, dtype: int64

### Exporting data

In [31]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_w_flags.pkl'))